# **Análise de Atrasos em Voos nos EUA**

Este projeto tem como objetivo construir um pipeline de dados utilizando a plataforma Databricks e ferramentas locais como Python, Jupyter e GitHub. A proposta é investigar padrões de atrasos aéreos nos Estados Unidos a partir de um conjunto de dados público disponibilizado pelo Kaggle. 

A análise será orientada pelas perguntas de negócio previamente definidas no README e executada de forma documentada, com foco em atender todos os critérios de avaliação propostos no desenvolvimento do MVP.


## **Setup Inicial**

In [17]:
# Importação inicial das bibliotecas essenciais para análise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de exibição
pd.set_option('display.max_columns', 100)
sns.set(style="whitegrid")


## **Carregamento dos Dados**

O notebook utiliza o arquivo `flights_amostra.csv`, uma versão reduzida com 20.000 registros aleatórios, gerada a partir do dataset original `flights.csv` (aproximadamente 580 MB), disponibilizado publicamente no Kaggle, que foi utilizado apenas localmente.

Devido às limitações de tamanho do GitHub, o arquivo completo não foi incluído no repositório.  
A amostra é suficiente para todas as análises previstas no projeto e está disponível para execução reprodutível.

Abaixo está o código utilizado para carregar a amostra.  
O trecho responsável pela geração do arquivo está comentado, pois foi executado previamente.


In [18]:
# A linha abaixo foi executada localmente e não é necessária para rodar o notebook da banca:
# df = pd.read_csv('data/flights.csv', low_memory=False)
# amostra = df.sample(n=20000, random_state=42)
# amostra.to_csv("data/flights_amostra.csv", index=False)

# Carregando a versão reduzida (disponível no repositório)
df = pd.read_csv('data/flights_amostra.csv', low_memory=False)

# Visualizando as primeiras linhas
df.head()


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,4,7,2,EV,4900,N759EV,FWA,DTW,1340,1335.0,-5.0,13.0,1348.0,56.0,48.0,30.0,128,1418.0,5.0,1436,1423.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,24,6,AS,611,N413AS,LAS,SEA,1910,1858.0,-12.0,14.0,1912.0,155.0,155.0,131.0,867,2123.0,10.0,2145,2133.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,7,8,3,WN,1483,N463WN,OAK,SEA,630,626.0,-4.0,8.0,634.0,110.0,106.0,92.0,672,806.0,6.0,820,812.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,5,26,2,WN,193,N7745A,STL,DAL,810,1043.0,153.0,6.0,1049.0,100.0,99.0,85.0,546,1214.0,8.0,950,1222.0,152.0,0,0,NaN,0.0,0.0,0.0,152.0,0.0
4,2015,7,6,1,UA,253,N213UA,IAH,HNL,1000,1008.0,8.0,16.0,1024.0,498.0,488.0,461.0,3904,1305.0,11.0,1318,1316.0,-2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## **Dimensões do Conjunto de Dados e Tipos de Variáveis**

Abaixo verifico a quantidade de linhas e colunas, além dos tipos de dados presentes. Isso me ajuda a entender a complexidade e possíveis etapas de tratamento necessárias.


In [19]:
print(f"Número de linhas: {df.shape[0]}")
print(f"Número de colunas: {df.shape[1]}")

print("\nInfo Amostra:")
df.info()

Número de linhas: 20000
Número de colunas: 31

Info Amostra:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   YEAR                 20000 non-null  int64  
 1   MONTH                20000 non-null  int64  
 2   DAY                  20000 non-null  int64  
 3   DAY_OF_WEEK          20000 non-null  int64  
 4   AIRLINE              20000 non-null  object 
 5   FLIGHT_NUMBER        20000 non-null  int64  
 6   TAIL_NUMBER          19948 non-null  object 
 7   ORIGIN_AIRPORT       20000 non-null  object 
 8   DESTINATION_AIRPORT  20000 non-null  object 
 9   SCHEDULED_DEPARTURE  20000 non-null  int64  
 10  DEPARTURE_TIME       19722 non-null  float64
 11  DEPARTURE_DELAY      19722 non-null  float64
 12  TAXI_OUT             19708 non-null  float64
 13  WHEELS_OFF           19708 non-null  float64
 14  SCHEDULED_TIME       2000

## **Reclassificação e Tratamento de Variáveis**

Após a verificação inicial da estrutura do dataset, identifiquei algumas colunas que, embora estejam tipadas como numéricas, não devem ser tratadas como variáveis quantitativas. Para garantir a integridade das análises estatísticas e facilitar o entendimento dos dados, realizarei os seguintes tratamentos:

- `FLIGHT_NUMBER`: representa o identificador do voo e não possui valor numérico significativo. Será convertido para `string`.

- `YEAR`: coluna com valor constante (2015), não contribui para análise e será removida.

- `CANCELLED` e `DIVERTED`: variáveis binárias que indicam status (0 = não, 1 = sim). Serão convertidas para o tipo `category`.

- `DAY`, `MONTH`, `DAY_OF_WEEK`: representam informações de calendário. Embora numéricas, não são contínuas e também serão convertidas para `category`.

- `ARRIVAL_TIME`, `WHEELS_ON`, `SCHEDULED_ARRIVAL`, `WHEELS_OFF`, `DEPARTURE_TIME`, `SCHEDULED_DEPARTURE`: estão codificadas como inteiros no formato `HHMM`. Para possibilitar análises temporais adequadas e evitar distorções estatísticas, essas colunas serão convertidas para o tipo `datetime.time`, utilizando `pd.to_datetime` com o formato `%H%M`.

Todas as transformações são fundamentais para garantir uma análise estatística coerente e alinhada ao propósito do projeto.


In [20]:
# Remoção da coluna constante
df.drop(columns=['YEAR'], inplace=True)

# Conversão de identificador para string
df['FLIGHT_NUMBER'] = df['FLIGHT_NUMBER'].astype(str)

# Conversão de variáveis categóricas
cols_categoricas = ['CANCELLED', 'DIVERTED', 'DAY', 'MONTH', 'DAY_OF_WEEK']
df[cols_categoricas] = df[cols_categoricas].astype('category')

# Conversão de horários no formato HHMM para datetime.time
colunas_horario = [
    'ARRIVAL_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL',
    'WHEELS_OFF', 'DEPARTURE_TIME', 'SCHEDULED_DEPARTURE'
]

for coluna in colunas_horario:
    df[coluna] = pd.to_datetime(df[coluna], format='%H%M', errors='coerce').dt.time


## **Verificação de Valores Ausentes**

Agora avalio a presença de valores nulos no dataset.  
Essa verificação é essencial para identificar colunas que podem demandar tratamento ou exclusão, e orientar a estratégia de preparação dos dados.


In [21]:
# Verificando valores ausentes
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Valores Nulos': missing_values,
    'Porcentagem (%)': missing_percent.round(2)
})

# Exibindo apenas colunas com valores nulos
missing_df[missing_df['Valores Nulos'] > 0].sort_values(by='Porcentagem (%)', ascending=False)


,Valores Nulos,Porcentagem (%)
CANCELLATION_REASON,19706,98.53
SECURITY_DELAY,16267,81.34
WEATHER_DELAY,16267,81.34
AIR_SYSTEM_DELAY,16267,81.34
LATE_AIRCRAFT_DELAY,16267,81.34
AIRLINE_DELAY,16267,81.34
ARRIVAL_TIME,389,1.94
WHEELS_ON,383,1.92
ELAPSED_TIME,337,1.68
ARRIVAL_DELAY,337,1.68


### **Análise de Valores Ausentes**

Com base na análise de valores nulos, observei que:

- A coluna `CANCELLATION_REASON` possui mais de 98% de valores ausentes, o que é esperado, já que a maioria dos voos **não foi cancelada**;
- As colunas relacionadas a atrasos (`AIR_SYSTEM_DELAY`, `SECURITY_DELAY`, `WEATHER_DELAY`, etc.) têm cerca de 81% de valores nulos, o que também é coerente, pois esses campos só são preenchidos **em caso de atraso**;
- Há colunas como `ARRIVAL_DELAY`, `AIR_TIME`, `TAXI_IN`, `DEPARTURE_TIME` com cerca de 1,5% de valores nulos — essas poderão demandar tratamento ou exclusão conforme o impacto na análise;
- Apenas a coluna `TAIL_NUMBER` apresenta uma taxa baixa de nulos (0,26%) e pode ser mantida sem grandes implicações.

Essas informações serão consideradas na etapa de tratamento de dados.


## **Estatísticas Descritivas**

Nesta etapa, aplico a função `describe()` para obter um resumo estatístico das variáveis numéricas do dataset.  
Esse resumo nos fornece informações importantes como média, desvio padrão, valores mínimo e máximo, e percentis, ajudando a identificar a presença de outliers e a dispersão dos dados.  
Esses insights são fundamentais para orientar as futuras etapas de limpeza e modelagem, garantindo a robustez das análises.


In [22]:
# Estatísticas descritivas para variáveis numéricas
df.describe().T.sort_values(by="std", ascending=False)


,count,mean,std,min,25%,50%,75%,max
DISTANCE,20000.0,824.611350,610.187120,31.0,373.0,647.0,1067.0,4983.0
SCHEDULED_TIME,20000.0,142.163650,75.575541,20.0,86.0,123.0,174.0,680.0
ELAPSED_TIME,19663.0,137.394853,74.441223,22.0,83.0,119.0,169.0,676.0
AIR_TIME,19663.0,113.824137,72.464148,9.0,61.0,95.0,144.0,647.0
AIRLINE_DELAY,3733.0,17.882668,44.855158,0.0,0.0,2.0,18.0,947.0
LATE_AIRCRAFT_DELAY,3733.0,22.861773,41.284678,0.0,0.0,3.0,29.0,470.0
ARRIVAL_DELAY,19663.0,4.190002,37.137156,-62.0,-13.0,-5.0,8.0,947.0
DEPARTURE_DELAY,19722.0,9.155968,34.781251,-36.0,-5.0,-2.0,8.0,965.0
AIR_SYSTEM_DELAY,3733.0,13.065631,24.811103,0.0,0.0,2.0,18.0,292.0
WEATHER_DELAY,3733.0,2.803375,16.899962,0.0,0.0,0.0,0.0,302.0


### **Interpretação das Estatísticas Descritivas**

A análise estatística das variáveis numéricas revelou alguns padrões importantes:

- As colunas com maior desvio padrão são `FLIGHT_NUMBER`, `DISTANCE` e variáveis de horário (`ARRIVAL_TIME`, `WHEELS_ON`, `SCHEDULED_ARRIVAL`), o que é esperado dado a variabilidade natural entre voos e rotas;
- Variáveis relacionadas a atrasos (`AIRLINE_DELAY`, `ARRIVAL_DELAY`, `DEPARTURE_DELAY`) apresentam **altos valores máximos**, sugerindo a presença de **outliers significativos**;
- As colunas `DEPARTURE_TIME`, `WHEELS_OFF`, `SCHEDULED_DEPARTURE` também apresentam alta dispersão, coerente com diferentes horários de operação;
- As colunas `CANCELLED` e `DIVERTED` têm médias próximas de zero, indicando baixa incidência desses eventos;
- A coluna `YEAR` é constante, com valor 2015, e pode ser descartada sem prejuízo para a análise;
- Algumas colunas como `SECURITY_DELAY` têm média e desvio padrão extremamente baixos, sugerindo baixa relevância analítica.

Essas observações são importantes para futuras etapas de:
- **Tratamento de outliers**
- **Remoção de colunas irrelevantes**
- **Normalização ou padronização, se necessário**
